In [2]:
import pandas as pd
import scipy.stats as st

In [3]:
data=pd.read_csv("/home/laksh/Downloads/bmi.csv")
data

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3
...,...,...,...,...
495,Female,150,153,5
496,Female,184,121,4
497,Female,141,136,5
498,Male,150,95,5


In [4]:
#Add a new status Coloumn based on Index
def index_to_description(x):
    if x['Index'] == 0:
        return 'Extremely Weak'
    elif x['Index'] == 1:
        return 'Weak'
    elif x['Index'] == 2:
        return 'Normal'
    elif x['Index'] == 3:
        return 'Overweight'
    elif x['Index']== 4:
        return 'Obesity'
    elif x['Index'] == 5:
        return 'Extreme Obesity'
data['Status'] = data.apply(index_to_description,axis=1)
data

,Gender,Height,Weight,Index,Status
0,Male,174,96,4,Obesity
1,Male,189,87,2,Normal
2,Female,185,110,4,Obesity
3,Female,195,104,3,Overweight
4,Male,149,61,3,Overweight
...,...,...,...,...,...
495,Female,150,153,5,Extreme Obesity
496,Female,184,121,4,Obesity
497,Female,141,136,5,Extreme Obesity
498,Male,150,95,5,Extreme Obesity


************T-Test**************

In [5]:
#Hypothesis T test for 1 sample
ttest,pVal = st.ttest_1samp(data.Height,169)
print("pValue = ",pVal)
if pVal < 0.05:
    print("Reject Null Hypothesis")
else:
    print("Reject Alternative Hypothesis")

pValue =  0.19797896234788953
Reject Alternative Hypothesis


In [6]:
#Hypothesis T test for 2 sample
ttest,pVal = st.ttest_ind(data.Height, data.Weight)
print("pValue = ",pVal)
if pVal < 0.05:
    print("Reject Null Hypothesis")
else:
    print("Reject Alternative Hypothesis")

pValue =  1.4541168650389354e-205
Reject Null Hypothesis


***************CHi Sqauare Testing***************

In [7]:

dataset=pd.crosstab(data['Gender'],data['Status'])
print(dataset)

Status  Extreme Obesity  Extremely Weak  Normal  Obesity  Overweight  Weak
Gender                                                                    
Female               93               7      41       71          36     7
Male                105               6      28       59          32    15


In [8]:
observed=dataset.values
print("Observed Values")
print(observed)

Observed Values
[[ 93   7  41  71  36   7]
 [105   6  28  59  32  15]]


In [9]:
val=st.chi2_contingency(dataset)
val[3]
expected=val[3]
print("Expected values")
print(expected)

Expected values
[[100.98   6.63  35.19  66.3   34.68  11.22]
 [ 97.02   6.37  33.81  63.7   33.32  10.78]]


In [10]:
#Calculate ddof
no_of_rows=len(dataset.iloc[0:2,0])
no_of_columns=len(data.iloc[0,0:6])
ddof=(no_of_rows-1)*(no_of_columns-1)
alpha = 0.05

In [11]:
#calculate Chi_Square
chi=sum([(o-e)**2./e for o,e in zip(observed,expected)])
chi_square=chi[0]+chi[1]+chi[2]+chi[3]+chi[4]+chi[5]
print(chi_square)

7.308471889700805


In [12]:
#Test Using Critical Value
critical_value=st.chi2.ppf(q=1-alpha,df=ddof) #percent point function
print('critical_value:',critical_value)
if chi_square>=critical_value:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")

critical_value: 9.487729036781154
Retain H0,There is no relationship between 2 categorical variables


In [13]:
#Test using P value
p_value=1-st.chi2.cdf(x=chi_square,df=ddof)
print('p-value:',p_value)
if p_value<=alpha:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")

p-value: 0.12045751220490897
Retain H0,There is no relationship between 2 categorical variables


*********Anova Test(F-Test)*************

In [14]:
data

,Gender,Height,Weight,Index,Status
0,Male,174,96,4,Obesity
1,Male,189,87,2,Normal
2,Female,185,110,4,Obesity
3,Female,195,104,3,Overweight
4,Male,149,61,3,Overweight
...,...,...,...,...,...
495,Female,150,153,5,Extreme Obesity
496,Female,184,121,4,Obesity
497,Female,141,136,5,Extreme Obesity
498,Male,150,95,5,Extreme Obesity


In [15]:
#One Way Anova Test
data1=data[['Weight','Status']]
status_type=pd.unique(data1.Status.values)
print(status_type)

['Obesity' 'Normal' 'Overweight' 'Extreme Obesity' 'Weak' 'Extremely Weak']


In [16]:
annova_data={grp:data1['Weight'][data1.Status == grp] for grp in status_type}


In [17]:
F_value, pval =st.f_oneway(annova_data['Obesity'],annova_data['Normal'],annova_data['Overweight'],annova_data['Extreme Obesity'],annova_data['Weak'],annova_data['Extremely Weak'])
print(pval)
if pval<0.05:
    print("reject null hypothesis")
else:
    print("accept null hypothesis")

3.1256254990187313e-115
reject null hypothesis


In [18]:
#2 Way Anova Testing

In [19]:
#Degree Of Freedom
n= len(data.Weight)
dof1 = len(data.Status.unique()) - 1
dof2 = len(data.Gender.unique()) - 1
dof3 = dof1*dof2 
dof4 = n - (len(data.Status.unique())*len(data.Gender.unique()))

In [20]:
weight_mean = data['Weight'].mean()

In [21]:
#Sum of Squares
square_gender = sum([(data[data.Gender ==l].Weight.mean()-weight_mean)**2 for l in data.Gender])
square_status = sum([(data[data.Status ==l].Weight.mean()-weight_mean)**2 for l in data.Status])
square_weight = sum((data.Weight - weight_mean)**2)
print(square_gender)
print(square_status)


47.45098039215665
348620.09687053406


In [30]:
male = data[data.Gender == 'Male']
female = data[data.Gender == 'Female']
male_means = [male[male.Gender == d].Weight.mean() for d in male.Gender]
female_means = [female[female.Gender == d].Weight.mean() for d in female.Gender]
square_within = sum((female.Weight - female_means)**2) +sum((male.Weight - male_means)**2)
square_both = square_weight - square_status -square_gender- square_within
print(square_both)
print(square_within)

-348620.0968705342
523220.54901960795


In [23]:
#Mean Of Square
mean_status =square_status/dof1
mean_gender =square_gender/dof2
mean_both =square_both/dof3
mean_within =square_within/dof4

In [24]:
#f_ratio
f_ratios= mean_status/mean_within
f_ratiog = mean_gender/mean_within
f_ratiob = mean_both/mean_within

In [25]:
#p-value
p_values = st.f.sf(f_ratios, dof1, dof4)
p_valueg = st.f.sf(f_ratiog, dof2,dof4)
p_valueb = st.f.sf(f_ratiob, dof3, dof4)

In [26]:
results = {'F_Ratio':[f_ratios, f_ratiog, f_ratiob],
            'P_Values':[p_values, p_valueg, p_valueb]}
columns=['F_Ratio', 'P_Values']

anova_result= pd.DataFrame(results, columns=columns,
                          index=['Status', 'Gender', 
                          'Gender:Status'])
anova_result

,F_Ratio,P_Values
Status,65.030553,5.755301e-52
Gender,0.044257,8.334643e-01
Gender:Status,-65.030553,1.000000e+00


In [27]:
#  Spearman Correlation
cor, pval = st.spearmanr(data.Weight, data.Height)
print('sperman')
print(cor, pval)
#  Pearson Correlation
cor, pval = st.pearsonr(data.Weight, data.Height)
print('pearson')
print(cor,pval)

sperman
-0.0012863887864474118 0.9771098018278506
pearson
0.0004459451283053379 0.9920638293471009


In [28]:
#Normalization 
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
mms.fit_transform(data[['Height','Weight']])

array([[0.57627119, 0.41818182],
       [0.83050847, 0.33636364],
       [0.76271186, 0.54545455],
       [0.93220339, 0.49090909],
       [0.15254237, 0.1       ],
       [0.83050847, 0.49090909],
       [0.11864407, 0.38181818],
       [0.23728814, 0.55454545],
       [0.57627119, 0.36363636],
       [0.49152542, 0.48181818],
       [0.93220339, 0.28181818],
       [0.3220339 , 0.27272727],
       [0.88135593, 0.46363636],
       [0.25423729, 0.00909091],
       [0.86440678, 0.26363636],
       [0.22033898, 0.51818182],
       [0.28813559, 0.54545455],
       [0.        , 0.71818182],
       [0.06779661, 0.86363636],
       [0.54237288, 0.80909091],
       [0.28813559, 0.54545455],
       [0.22033898, 0.9       ],
       [0.49152542, 0.42727273],
       [0.76271186, 0.80909091],
       [0.54237288, 0.15454545],
       [0.18644068, 0.12727273],
       [0.84745763, 0.40909091],
       [0.79661017, 0.10909091],
       [0.38983051, 0.99090909],
       [0.66101695, 0.92727273],
       [0.

In [29]:
#standardization
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit_transform(data[['Height','Weight']])

array([[ 0.24793878, -0.30911702],
       [ 1.16487212, -0.58732233],
       [ 0.92035656,  0.12364681],
       [ 1.53164546, -0.0618234 ],
       [-1.28028346, -1.39102658],
       [ 1.16487212, -0.0618234 ],
       [-1.40254124, -0.43276382],
       [-0.97463902,  0.15455851],
       [ 0.24793878, -0.49458723],
       [-0.05770567, -0.09273511],
       [ 1.53164546, -0.77279254],
       [-0.66899457, -0.80370424],
       [ 1.34825879, -0.15455851],
       [-0.91351013, -1.70014359],
       [ 1.2871299 , -0.83461595],
       [-1.03576791,  0.0309117 ],
       [-0.79125235,  0.12364681],
       [-1.83044347,  0.71096914],
       [-1.58592791,  1.20555637],
       [ 0.125681  ,  1.02008616],
       [-0.79125235,  0.12364681],
       [-1.03576791,  1.32920317],
       [-0.05770567, -0.27820532],
       [ 0.92035656,  1.02008616],
       [ 0.125681  , -1.20555637],
       [-1.15802568, -1.29829147],
       [ 1.22600101, -0.34002872],
       [ 1.04261434, -1.36011487],
       [-0.42447901,